In [1]:
def solve_cryptarithmetic(puzzle):
    words = puzzle[:-1].split('+')
    result_word = puzzle[-1]
    chars = set(''.join(words) + result_word)

    def is_valid_assignment(assignment):
        return sum(assignment[word[0]] for word in words) == assignment[result_word[0]]

    def is_consistent(char, digit, assignment):
        return char not in assignment or assignment[char] == digit

    def propagate_constraints(assignment):
        updated = True
        while updated:
            updated = False
            for word in words:
                if len(word) == 1:  # Si la palabra tiene solo un carácter, no se pueden propagar restricciones
                    continue
                if all(c in assignment for c in word[:-1]):
                    last_char = word[-1]
                    if last_char not in assignment:
                        possible_digits = [int(d) for d in range(10) if d not in assignment.values()]
                        consistent_digits = [d for d in possible_digits if is_consistent(last_char, d, assignment)]
                        if len(consistent_digits) == 1:
                            assignment[last_char] = consistent_digits[0]
                            updated = True

    def select_next_char(assignment):
        unassigned_chars = chars - assignment.keys()
        unassigned_chars = sorted(unassigned_chars, key=lambda char: calculate_degree(char), reverse=True)
        return unassigned_chars[0] if unassigned_chars else None

    def calculate_degree(char):
        return sum(1 for word in words if char in word)

    def backtrack_search(assignment):
        if len(assignment) == len(chars):
            if is_valid_assignment(assignment):
                return assignment
            else:
                return None

        char = select_next_char(assignment)
        if char is None:
            return None

        for digit in range(10):
            if is_consistent(char, digit, assignment):
                assignment[char] = digit
                propagate_constraints(assignment)  # Propagar restricciones después de cada asignación
                result = backtrack_search(assignment)
                if result:
                    return result
                assignment.pop(char)
        return None

    solution = backtrack_search({})
    return solution

# Example usage
if __name__ == "__main__":
    puzzle = "SEND+MORE=MONEY"
    solution = solve_cryptarithmetic(puzzle)
    if solution:
        print("Solución encontrada:")
        for char, digit in solution.items():
            print(f"{char}: {digit}")
    else:
        print("No se encontró solución.")

Solución encontrada:
N: 0
E: 0
D: 0
O: 0
R: 0
S: 0
=: 0
M: 0
Y: 0


In [10]:
equation = "SEND+MORE=MONEY"
solution = solve_cryptarithmetic(equation)

if solution:
  print("Solución encontrada:")
  for char, digit in solution.items():
    print(f"{char}: {digit}")
else:
  print("No se encontró solución.")


Solución encontrada:
Y: 0
N: 0
D: 0
E: 0
O: 0
R: 0
S: 0
=: 0
M: 0


In [18]:
import random 

In [ ]:
#### 

In [48]:
class Variable:
    def __init__(self, name):
        self.name = name
        self.domain = set(range(10))
        self._assigned = False

    def assigned(self):
        return self._assigned

    def unassign(self):
        self._assigned = False

    def assign(self, value):
        self.domain = {value}
        self._assigned = True
        

class Constraint:
    def __init__(self, variable1, position, value):
        self.variable1 = variable1
        self.position = position
        self.value = value
        self.domain1 = set(range(10))
        self.domain2 = set(range(10))

    def reduce_domain(self, variable2):
        if variable2 in self.variable1.domain:
            self.domain1.intersection_update(self.variable1.domain)
            self.domain2.intersection_update(self.variable1.domain)

    def reduce_domain1_intersection(self):
        self.domain1.intersection_update(self.variable1.domain)

    def reduce_domain2_intersection(self):
        self.domain2.intersection_update(self.variable1.domain)
        
class Cryptarithmetic:
    def __init__(self, equation):
        self.equation = equation
        self.variables = {}
        self.domains = {}
        self.constraints = {}
        self.assignments = {}
        self.propagate_rules = []

        # Define the variables
        for letter in equation:
            if letter.isalpha() and letter not in self.assignments:
                self.variables[letter] = Variable(letter)
                self.domains[letter] = set(range(10))

        # Define the constraints
        for i, (a, b, c) in enumerate(zip(equation[:-1], equation[1:], equation[2:])):
            if a.isalpha() and b.isalpha() and c.isdigit():
                self.constraints[(a, i)] = Constraint(a, i, int(c) - int(b))

        # Define the propagation rules
        self.propagate_rules.append(self.rule_unique_value)
        self.propagate_rules.append(self.rule_sum_domain)
        self.propagate_rules.append(self.rule_difference_domain)

    def rule_unique_value(self):
        for variable in self.variables.values():
            if len(variable.domain) == 1:
                self.assignments[variable.name] = variable.domain.pop()
                for constraint in self.constraints.values():
                    constraint.reduce_domain(variable.name)

    def rule_sum_domain(self):
        for constraint in self.constraints.values():
            if len(constraint.domain1.intersection(constraint.domain2)) == 0:
                return
            if constraint.value <= sum(constraint.domain1.intersection(constraint.domain2)):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def rule_difference_domain(self):
        for constraint in self.constraints.values():
            if max(constraint.domain1) - min(constraint.domain2) > constraint.value - sum(constraint.domain2):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def propagate(self):
        for rule in self.propagate_rules:
            rule()

    def consistent(self):
        for variable in self.variables.values():
            if len(variable.domain) == 0:
                return False
        return True

    def solved(self):
        return len(self.assignments) == len(self.variables)
    
    def search(self, depth_limit=1):
        if not self.consistent():
            return False
        if self.solved():
            self.solutions.append(self.assignments.copy())
            return True

        variable = min(self.variables.values(), key=lambda x: len(x.domain))
        if len(variable.domain) == 0:
            return False

        for value in variable.domain:
            variable.assign(value)
            self.propagate()
            if self.consistent():
                if self.search(depth_limit + 1):
                    return True
            self.backtrack()

        variable.unassign()
        return False

    def backtrack(self):
        for variable in self.variables.values():
            if variable.assigned():
                variable.unassign()
                self.propagate()
                if not self.consistent():
                    return False
        return True

In [49]:
eq = "SEND + MORE = MONEY"
crypt = Cryptarithmetic(eq)
crypt.search()

for variable, value in crypt.assignments.items():
    print(f"{variable} = {value}")

S = 9


In [44]:
class Variable:
    def __init__(self, name):
        self.name = name
        self.domain = set(range(10))
        self._assigned = False

    def assigned(self):
        return self._assigned

    def unassign(self):
        self._assigned = False

    def assign(self, value):
        self.domain = {value}
        self._assigned = True

class Constraint:
    def __init__(self, variable1, position, value):
        self.variable1 = variable1
        self.position = position
        self.value = value
        self.domain1 = set(range(10))
        self.domain2 = set(range(10))

    def reduce_domain(self, variable2):
        if variable2 in self.variable1.domain:
            self.domain1.intersection_update(self.variable1.domain)
            self.domain2.intersection_update(self.variable1.domain)

    def reduce_domain1_intersection(self):
        self.domain1.intersection_update(self.variable1.domain)

    def reduce_domain2_intersection(self):
        self.domain2.intersection_update(self.variable1.domain)

class Cryptarithmetic:
    def __init__(self, equation):
        self.equation = equation
        self.variables = {}
        self.domains = {}
        self.constraints = {}
        self.assignments = {}
        self.solutions = []  # List to store all solutions
        self.propagate_rules = []

        # Define the variables
        for letter in equation:
            if letter.isalpha() and letter not in self.assignments:
                self.variables[letter] = Variable(letter)
                self.domains[letter] = set(range(10))

        # Define the constraints (use dictionary comprehension for efficiency)
        self.constraints = {
            (a, i): Constraint(a, i, int(c) - int(b))
            for i, (a, b, c) in enumerate(zip(equation[:-1], equation[1:], equation[2:]))
            if a.isalpha() and b.isalpha() and c.isdigit()
        }

        # Define the propagation rules
        self.propagate_rules.append(self.rule_unique_value)
        self.propagate_rules.append(self.rule_sum_domain)
        self.propagate_rules.append(self.rule_difference_domain)

    def rule_unique_value(self):
        for variable in self.variables.values():
            if len(variable.domain) == 1:
                self.assignments[variable.name] = variable.domain.pop()
                for constraint in self.constraints.values():
                    constraint.reduce_domain(variable.name)

    def rule_sum_domain(self):
        for constraint in self.constraints.values():
            if len(constraint.domain1.intersection(constraint.domain2)) == 0:
                return
            if constraint.value <= sum(min(constraint.domain1), min(constraint.domain2)):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def rule_difference_domain(self):
        for constraint in self.constraints.values():
            if max(constraint.domain1) - min(constraint.domain2) > constraint.value - sum(constraint.domain2):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def propagate(self):
        for rule in self.propagate_rules:
            rule()

    def consistent(self):
        for variable in self.variables.values():
            if len(variable.domain) == 0:
                return False
        return True

    def solved(self):
        return len(self.assignments) == len(self.variables)

    def search(self, depth_limit=1):
        if not self.consistent():
            return False
        if self.solved():
            self.solutions.append(self.assignments.copy())
            return True

        variable = min(self.variables.values(), key=lambda x: len(x.domain))
        if len(variable.domain) == 0:
            return False

        for value in variable.domain:
            variable.assign(value)
            self.propagate()
            if self.consistent():
                # Recursive call to explore this assignment further
                if self.search(depth_limit + 1):
                    return True
        # Backtrack: undo the assignment and try other values
            self.backtrack()

        variable.unassign()
        return False

    def backtrack(self):
        for variable in self.variables.values():
            if variable.assigned():
                variable.unassign()
                self.propagate()
                if not self.consistent():
                    return False
        return True


In [45]:
eq = "SEND + MORE = MONEY"
crypt = Cryptarithmetic(eq)
solutions = crypt.search()

if solutions:
    for solution in solutions:
        for var, value in solution.items():
            print(f"{var} = {value}")
        print("---")
else:
    print("No solutions found!")


No solutions found!
